# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.


# 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
import re
import pickle
import numpy as np
import pandas as pd

from sqlalchemy import create_engine

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# punkt is a tokenizer that is used to split text into individual words and sentences.
# wordnet is a lexical database of English words that is used for 
# various NLP tasks such as stemming, lemmatization, and synonym detection.
# nltk.download(['punkt', 'wordnet']);

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator,TransformerMixin

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
# load data from database
engine = create_engine('sqlite:///ETL_Preparation.db')
df = pd.read_sql_table('ETL_Preparation', engine)
df.head(3)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df.groupby('related').count()

,id,message,original,genre,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
related,,,,,,,,,,,,,,,,,,,,,
0,6122,6122,3395,6122,6122,6122,6122,6122,6122,6122,...,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122
1,19906,19906,6643,19906,19906,19906,19906,19906,19906,19906,...,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906


In [5]:
#  this code is to replace any occurrence of the value 2 in the related column with the value 1.
# because the value 2 'so small' in the 'related' column
df['related']=df['related'].map(lambda x: 1 if x == 2 else x)
df.groupby('related').count() # 19906 + 188 = 20094

,id,message,original,genre,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
related,,,,,,,,,,,,,,,,,,,,,
0,6122,6122,3395,6122,6122,6122,6122,6122,6122,6122,...,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122
1,19906,19906,6643,19906,19906,19906,19906,19906,19906,19906,...,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906


In [6]:
# Extract X and y variables from the data for the modelling
X = df['message']
y = df.iloc[:,4:]


# 2. Write a tokenization function to process your text data

In [7]:
def tokenize(text, url_place_holder_string="urlplaceholder"):
    
    # Replace all urls with a urlplaceholder string
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # Extract all the urls from the provided text 
    detected_urls = re.findall(url_regex, text)
    
    # Replace url with a url placeholder string
    for detected_url in detected_urls:
        text = text.replace(detected_url, url_place_holder_string)

    # Extract the word tokens from the provided text
    tokens = nltk.word_tokenize(text)
    
    # 'Lemmanitizer' is used to reduce words to the 'root word'
    lemmatizer = nltk.WordNetLemmatizer()

    # List of clean tokens
    clean_tokens = [lemmatizer.lemmatize(w).lower().strip() for w in tokens]
    return clean_tokens

In [8]:
# The InitialVerbDetector class is a custom scikit-learn transformer that identifies if any sentence
# within a given text starts with a verb. It inherits from BaseEstimator and TransformerMixin and
# implements the fit() and transform() methods. The main function, detect_initial_verb(), tokenizes
# the input text into sentences and then checks the part-of-speech (POS) tag of the first word in each
# sentence. If the POS tag is 'VB' (base form verb) or 'VBP' (non-3rd person singular present verb),
# or if the first word is 'RT' (usually indicating a retweet on Twitter), it returns True. Otherwise,
# it returns False. This transformer can be useful in various natural language processing tasks, such
# as sentiment analysis or text classification.

class InitialVerbDetector(BaseEstimator, TransformerMixin):

    def detect_initial_verb(self, input_text):
        sentences = nltk.sent_tokenize(input_text)

        for sentence in sentences:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            initial_word, initial_tag = pos_tags[0]

            if initial_tag in ['VB', 'VBP'] or initial_word == 'RT':
                return True

        return False

    def transform(self, input_data):
        tagged_data = pd.Series(input_data).apply(self.detect_initial_verb)
        return pd.DataFrame(tagged_data)

    # Implements the scikit-learn transformer interface by returning 'self'
    def fit(self, input_data, target_data=None):
        return self

    

# 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
pipe1 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ]))
            
        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])


pipe2 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ])),

            ('starting_verb_transformer', InitialVerbDetector() )
        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])


pipeline using scikit-learn library. The pipeline consists of two main components:

- FeatureUnion: This component allows combining multiple feature extraction methods. In this example, it includes only one feature extraction method - text_pipeline.

- MultiOutputClassifier: This is a wrapper for multi-output classification algorithms. In this example, it uses the AdaBoostClassifier algorithm for multi-output classification.

The text_pipeline feature extraction method consists of two sub-components:

- CountVectorizer: This is used to convert a collection of text documents to a matrix of token counts.

- TfidfTransformer: This is used to transform a count matrix to a normalized term-frequency times inverse document-frequency representation.

The pipeline is designed to handle multi-output classification problems, meaning that it can predict multiple target variables at the same time. 


# 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline_fitted1 = pipe1.fit(X_train, y_train)
# joblib.dump(pipeline_fitted1, ppipeline_fitted1d.pkl') # save the 'pipeline_fitted' to a file

# 'train' making 'equations'
# 1754_SatApril2023, about 2 minutes
# 1915_SatApril2023, about 3 minutes
# 1056_Thu_15June2023, about 2 minutes

/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


# 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
# load the saved pipeline from file
# pipeline_fitted1 = joblib.load('pipeline_fitted1.pkl')

y_prediction_train = pipeline_fitted1.predict(X_train)
y_prediction_test = pipeline_fitted1.predict(X_test)

# Print classification report on test data
print(classification_report(y_test.values, y_prediction_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.83      0.94      0.88      4981
               request       0.77      0.54      0.64      1105
                 offer       0.00      0.00      0.00        30
           aid_related       0.77      0.59      0.67      2738
          medical_help       0.62      0.26      0.37       535
      medical_products       0.67      0.37      0.48       337
     search_and_rescue       0.38      0.13      0.19       154
              security       0.44      0.07      0.12       104
              military       0.57      0.31      0.41       181
           child_alone       0.00      0.00      0.00         0
                 water       0.76      0.63      0.69       410
                  food       0.84      0.69      0.76       756
               shelter       0.73      0.56      0.63       553
              clothing       0.78      0.48      0.59       103
                 money       0.56      

/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(a

# 6. Improve your model (Using GridSearchCV)
Use GridSearchCV search to find better parameters. 

In [12]:
# returns the keys (in python dictionary) of all the parameters in the pipeline
pipe1.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'features', 'classifier', 'features__n_jobs', 'features__transformer_list', 'features__transformer_weights', 'features__verbose', 'features__text_pipeline', 'features__text_pipeline__memory', 'features__text_pipeline__steps', 'features__text_pipeline__verbose', 'features__text_pipeline__count_vectorizer', 'features__text_pipeline__tfidf_transformer', 'features__text_pipeline__count_vectorizer__analyzer', 'features__text_pipeline__count_vectorizer__binary', 'features__text_pipeline__count_vectorizer__decode_error', 'features__text_pipeline__count_vectorizer__dtype', 'features__text_pipeline__count_vectorizer__encoding', 'features__text_pipeline__count_vectorizer__input', 'features__text_pipeline__count_vectorizer__lowercase', 'features__text_pipeline__count_vectorizer__max_df', 'features__text_pipeline__count_vectorizer__max_features', 'features__text_pipeline__count_vectorizer__min_df', 'features__text_pipeline__count_vectorizer__ngram_range', '

## Set up the 'parameter grid' for GridSearchCV

In [13]:
parameters_grid1 = {'classifier__estimator__learning_rate': [0.03],
              'classifier__estimator__n_estimators': [13]}


# 'f1_micro' is used as the evaluation metric, which is a common metric used for multiclass classification problems.
#  -1 is used to use all available CPU cores.
cv = GridSearchCV(pipe1, param_grid=parameters_grid1, scoring='f1_micro', n_jobs=-1)


In [14]:
cv

GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('count_vectorizer',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x7f307af09cf0>)),
                                                                                        ('tfidf_transformer',
                                                                                         TfidfTransformer())]))])),
                                       ('classifier',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             n_jobs=-1,
             param_grid={'classifier__estimator__learning_rate': [0.03],
                         'classifier__estimator__n_estimators': [13]},
             scoring='f1_micro')

In [15]:
# (cv) will contain the best hyperparameters found during the search, 
# and other information about the search process.
cv.fit(X_train, y_train)

# 1113_Thu_15June2023, 3 minutes

/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/feature_ex

GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('count_vectorizer',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x7f307af09cf0>)),
                                                                                        ('tfidf_transformer',
                                                                                         TfidfTransformer())]))])),
                                       ('classifier',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             n_jobs=-1,
             param_grid={'classifier__estimator__learning_rate': [0.03],
                         'classifier__estimator__n_estimators': [13]},
             scoring='f1_micro')

In [16]:
# showing the 'best parameters' found by the 'grid search'
print(cv.best_params_)

{'classifier__estimator__learning_rate': 0.03, 'classifier__estimator__n_estimators': 13}


## Updating the 'hyper-parameters' in the 'pipe1 model' with the 'best parameters' from GridSearchCV
For the 'AdaBoostClassifier'

In [17]:
pipe1 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ]))
            
        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier(learning_rate=0.03, n_estimators=13)))
    ])

# 'Fitting' the data again with 'pip1'

In [19]:
# splitting the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y)

# 'fitting' again
pipeline_fitted2 = pipe1.fit(X_train, y_train)

# 1129_Thu_15June2023, 39 seconds

/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


# Making 'prediction' again with the updated 'model' 'pipeline_fitted2'

In [ ]:
y_prediction_train = pipeline_fitted2.predict(X_train)
y_prediction_test = pipeline_fitted2.predict(X_test)

In [14]:
# y_prediction_test.shape # (6554, 36) rows, 36 columns
# y_prediction_train.shape # (19662, 36) rows, 36 columns

(19662, 36)

# 7. Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [21]:
# 'classification_report' for the 'test data'
print(classification_report(y_test.values, y_prediction_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.77      1.00      0.87      5014
               request       0.87      0.15      0.25      1172
                 offer       0.00      0.00      0.00        32
           aid_related       0.80      0.19      0.31      2722
          medical_help       0.75      0.01      0.01       540
      medical_products       1.00      0.01      0.01       324
     search_and_rescue       0.67      0.17      0.27       170
              security       0.00      0.00      0.00       117
              military       0.00      0.00      0.00       205
           child_alone       0.00      0.00      0.00         0
                 water       0.58      0.85      0.69       407
                  food       0.77      0.67      0.72       761
               shelter       0.83      0.31      0.45       571
              clothing       0.00      0.00      0.00       103
                 money       0.00      

/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, 

In [22]:
# 'classification_report' for the 'train data'
print(classification_report(y_train.values, y_prediction_train, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.76      1.00      0.87     14892
               request       0.85      0.14      0.25      3302
                 offer       0.00      0.00      0.00        86
           aid_related       0.80      0.19      0.30      8138
          medical_help       0.75      0.00      0.01      1544
      medical_products       0.71      0.01      0.01       989
     search_and_rescue       0.63      0.19      0.30       554
              security       0.00      0.00      0.00       354
              military       0.00      0.00      0.00       655
           child_alone       0.00      0.00      0.00         0
                 water       0.58      0.84      0.69      1265
                  food       0.77      0.68      0.72      2162
               shelter       0.85      0.29      0.43      1743
              clothing       1.00      0.01      0.01       302
                 money       1.00      

/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, 

# 8. Try improving your model further. (pipe2 'model')

Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

## using 'pipe2 model'

In [23]:
# using pipe2 which includes StartingVerbEstimator
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipe2_pipeline_fitted = pipe2.fit(X_train, y_train)

# 0306_MonApril2023, about 2 minutes

/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [24]:
# making prediction/guessing from 'pipe2'
y_prediction_from_train_data2 = pipe2_pipeline_fitted.predict(X_train)
y_prediction_from_test_data2 = pipe2_pipeline_fitted.predict(X_test)

In [25]:
# print the 'accuracy report' for the 'train data'
print(classification_report(y_train.values, y_prediction_from_train_data2, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.82      0.96      0.88     14965
               request       0.79      0.53      0.64      3322
                 offer       0.35      0.08      0.14        83
           aid_related       0.77      0.61      0.68      8145
          medical_help       0.66      0.28      0.40      1590
      medical_products       0.70      0.33      0.45      1000
     search_and_rescue       0.68      0.21      0.33       557
              security       0.54      0.10      0.17       376
              military       0.66      0.37      0.48       660
           child_alone       0.00      0.00      0.00         0
                 water       0.76      0.67      0.71      1243
                  food       0.83      0.69      0.75      2173
               shelter       0.80      0.57      0.66      1720
              clothing       0.78      0.44      0.56       302
                 money       0.62      

/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(a

In [26]:
# print the 'accuracy report' for the 'test data'
print(classification_report(y_test.values, y_prediction_from_test_data2, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.82      0.95      0.88      4941
               request       0.79      0.57      0.67      1152
                 offer       0.00      0.00      0.00        35
           aid_related       0.76      0.61      0.68      2715
          medical_help       0.62      0.28      0.38       494
      medical_products       0.67      0.36      0.47       313
     search_and_rescue       0.60      0.22      0.32       167
              security       0.15      0.04      0.07        95
              military       0.72      0.32      0.44       200
           child_alone       0.00      0.00      0.00         0
                 water       0.72      0.67      0.70       429
                  food       0.80      0.70      0.75       750
               shelter       0.75      0.57      0.65       594
              clothing       0.69      0.44      0.54       103
                 money       0.52      

/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(a

# 9. Export your model as a pickle file

In [27]:
import os
os.getcwd()

'/workspaces/disaster-response-pipeline-uda/Piplines'

In [28]:
# 'wb' write in 'binary mode'
with open('mo2.pkl', 'wb') as file:
    pickle.dump(cv, file)

# 10. Use this notebook to complete `models/train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.